in this notebook, I combined the data that is needed for our research project, using 76 countries in GPS as the key because this is our main interest in the project, the GDP, humanitarian aids data has been filtered to after 2003, for the reason that democratic index only starts from 2006.
notice that Bosnia Herzegovina have no record in humanitarian aids data.

### data output 
| Variable     |Type |      Content                                             |
|:-------------|:---:|:------------------------------------------------------- :|
| country   |  string| name of countries from Global Preference Survey(GPS) data|
| isocode | string|  isocode of the countries                             |
|2019 ~ 2006| float| democratic index by year, notice 2007 & 2009 doesn't exist|
|altruism| float| altruism from GPS | 
|2003_gdp ~ 2019_gdp|int | gdp in current US$ by year                           |
|funding_3 ~ funding_19| int| humanitarian aid funding (US$) by year       |
|pledge_3 ~ pledge_19| int | pledging humanitarian aid (US$) by year        |
|funding_3_pct~funding_19_pct| float | funding / gdp the year * 100           |
|pledge_3_pct ~ pledge_19_pct| float| pledging funding / gdp the year * 100    |


In [1]:
import pandas as pd
import glob
from auxiliary import *


### read in  democracy index and renaming some countries for merging later

In [4]:
democracy = pd.read_excel('data/EIU_Democracy_Index_2006_to_2019.xlsx').rename(columns={"Unnamed: 0": "country"})

In [5]:
country_dict = {'US': 'United States',  'Bosnia and Hercegovina' : 'Bosnia Herzegovina', 'UK' : 'United Kingdom', 'UAE': 'United Arab Emirates' }
democracy = democracy.replace({"country": country_dict})
democracy.head()

,country,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2008,2006
0,Canada,9.22,9.15,9.15,9.15,9.08,9.08,9.08,9.08,9.08,9.08,9.07,9.07
1,United States,7.96,7.96,7.98,7.98,8.05,8.11,8.11,8.11,8.11,8.18,8.22,8.22
2,Austria,8.29,8.29,8.42,8.41,8.54,8.54,8.48,8.62,8.49,8.49,8.49,8.69
3,Belgium,7.64,7.78,7.78,7.77,7.93,7.93,8.05,8.05,8.05,8.05,8.16,8.15
4,Cyprus,7.59,7.59,7.59,7.65,7.53,7.40,7.29,7.29,7.29,7.29,7.70,7.60


### read in Global Preference Survey (GPS) data and merge with  democracy index 

In [6]:
gps = pd.read_stata('data/country.dta')
gps.shape

(76, 8)

In [7]:
gps.head()

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust
0,Afghanistan,AFG,-0.201360,0.120764,0.289641,0.254712,0.166455,0.315964
1,Algeria,DZA,0.059815,0.391530,-0.598255,0.254901,-0.009968,-0.177371
2,Argentina,ARG,-0.229308,0.041503,0.159679,-0.140457,0.027347,-0.206733
3,Australia,AUS,0.657004,0.137137,0.069660,0.022190,0.155331,0.298973
4,Austria,AUT,0.608285,-0.061829,0.161047,-0.055415,-0.035363,0.154628


In [8]:
gps_democracy = pd.merge(gps, democracy, how = 'left', on = 'country')
gps_democracy.shape

(76, 20)

In [9]:
# gps_democracy[gps_democracy.isna().any(axis=1)]
gps_democracy.head()

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2008,2006
0,Afghanistan,AFG,-0.201360,0.120764,0.289641,0.254712,0.166455,0.315964,2.85,2.97,2.55,2.55,2.77,2.77,2.48,2.48,2.48,2.48,3.02,3.06
1,Algeria,DZA,0.059815,0.391530,-0.598255,0.254901,-0.009968,-0.177371,4.01,3.50,3.56,3.56,3.95,3.83,3.83,3.83,3.44,3.44,3.32,3.17
2,Argentina,ARG,-0.229308,0.041503,0.159679,-0.140457,0.027347,-0.206733,7.02,7.02,6.96,6.96,7.02,6.84,6.84,6.84,6.84,6.84,6.63,6.63
3,Australia,AUS,0.657004,0.137137,0.069660,0.022190,0.155331,0.298973,9.09,9.09,9.09,9.01,9.01,9.01,9.13,9.22,9.22,9.22,9.09,9.09
4,Austria,AUT,0.608285,-0.061829,0.161047,-0.055415,-0.035363,0.154628,8.29,8.29,8.42,8.41,8.54,8.54,8.48,8.62,8.49,8.49,8.49,8.69


### read in UN Humanitarian Affairs Financial Tracking Service data by year and rename columns for merging

In [10]:
file_name = 'data/OCHA_FTS_Government_Donations_20'
extension = '.xlsx'
all_filenames = [i for i in glob.glob(f'{file_name}*{extension}')]

In [11]:
for i in range(len(all_filenames)):
    new = pd.read_excel(all_filenames[i], sheet_name='Export data', skiprows=2).rename(columns={"Source org.": "country", "Funding US$": "funding", "Pledges US$" : "pledge"})
    if i == 0:
        df = new
    else:
        df = pd.merge(df, new, how='outer', on="country", suffixes=('', f'_{i+3}'))

donate = df.rename(columns={"pledge": "pledge_3", "funding": "funding_3"})

donate['country'] = donate['country'].str.rstrip(' Government of')
donate['country'] = donate['country'].str.rstrip(',')


In [12]:
#checking 2006 & 2020 manually 
country_dict = {'United States of America': 'United States',  'Saudi Arabia (Kingdom of)' : 'Saudi Arabia', 'Russian Federation' : 'Russia', 'Korea, Republic of': 'South Korea', 'Viet Nam' : 'Vietnam'}
donate = donate.replace({"country": country_dict})

### merge humanitarian data with previous dataframe, notice Bosnia does not get matched

In [13]:
gps_demo_donate = pd.merge(gps_democracy, donate,  how = 'left', on = 'country')
gps_demo_donate.shape

(76, 56)

In [14]:
gps_demo_donate.head()

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust,2019,2018,...,funding_16,pledge_16,funding_17,pledge_17,funding_18,pledge_18,funding_19,pledge_19,funding_20,pledge_20
0,Afghanistan,AFG,-0.201360,0.120764,0.289641,0.254712,0.166455,0.315964,2.85,2.97,...,NaN,NaN,NaN,NaN,3870542.0,0.0,903177.0,0.0,NaN,NaN
1,Algeria,DZA,0.059815,0.391530,-0.598255,0.254901,-0.009968,-0.177371,4.01,3.50,...,660000.0,0.0,10000.0,0.0,10000.0,0.0,NaN,NaN,0.0,0.0
2,Argentina,ARG,-0.229308,0.041503,0.159679,-0.140457,0.027347,-0.206733,7.02,7.02,...,176272.0,0.0,1344591.0,0.0,265000.0,0.0,155000.0,0.0,NaN,NaN
3,Australia,AUS,0.657004,0.137137,0.069660,0.022190,0.155331,0.298973,9.09,9.09,...,205207886.0,2724017.0,243900706.0,2712522.0,248119692.0,0.0,310639840.0,0.0,271692972.0,0.0
4,Austria,AUT,0.608285,-0.061829,0.161047,-0.055415,-0.035363,0.154628,8.29,8.29,...,35663228.0,0.0,49109105.0,0.0,26096928.0,6670328.0,32476153.0,1540154.0,19142494.0,0.0


In [16]:
gps_demo_donate[gps_demo_donate.filter(regex='(funding|pledge)_*', axis=1).isna().all(axis=1)]

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust,2019,2018,...,funding_16,pledge_16,funding_17,pledge_17,funding_18,pledge_18,funding_19,pledge_19,funding_20,pledge_20
7,Bosnia Herzegovina,BIH,-0.247258,-0.125639,0.096463,0.429355,-0.11531,-0.193106,4.86,4.98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### read in GDP data and merge with previous data frame

In [17]:
gdp = pd.read_excel('data/GDP_by_country_by_year.xls', sheet_name='Data', skiprows=3).rename(columns={"Country Code": "isocode"}).set_index('isocode')
gdp = gdp.iloc[:,-18 : -1]
gdp = gdp.add_suffix('_gdp')
gdp.reset_index()
gdp.head()


,2003_gdp,2004_gdp,2005_gdp,2006_gdp,2007_gdp,2008_gdp,2009_gdp,2010_gdp,2011_gdp,2012_gdp,2013_gdp,2014_gdp,2015_gdp,2016_gdp,2017_gdp,2018_gdp,2019_gdp
isocode,,,,,,,,,,,,,,,,,
ABW,2.021229e+09,2.228492e+09,2.330726e+09,2.424581e+09,2.615084e+09,2.745251e+09,2.498883e+09,2.390503e+09,2.549721e+09,2.534637e+09,2.701676e+09,2.765363e+09,2.919553e+09,2.965922e+09,3.056425e+09,NaN,NaN
AFG,4.515559e+09,5.226779e+09,6.209138e+09,6.971286e+09,9.747880e+09,1.010923e+10,1.243909e+10,1.585657e+10,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.936264e+10,2.019176e+10,1.948438e+10,1.910135e+10
AGO,1.781271e+10,2.355205e+10,3.697092e+10,5.238101e+10,6.526645e+10,8.853861e+10,7.030716e+10,8.379950e+10,1.117897e+11,1.280529e+11,1.367099e+11,1.457122e+11,1.161936e+11,1.011239e+11,1.221238e+11,1.013532e+11,9.463542e+10
ALB,5.611492e+09,7.184681e+09,8.052076e+09,8.896074e+09,1.067732e+10,1.288135e+10,1.204422e+10,1.192693e+10,1.289077e+10,1.231983e+10,1.277622e+10,1.322814e+10,1.138685e+10,1.186120e+10,1.301969e+10,1.514702e+10,1.527808e+10
AND,2.361727e+09,2.894922e+09,3.159905e+09,3.456442e+09,3.952601e+09,4.085631e+09,3.674410e+09,3.449967e+09,3.629204e+09,3.188809e+09,3.193704e+09,3.271808e+09,2.789870e+09,2.896679e+09,3.000181e+09,3.218316e+09,3.154058e+09


In [18]:
gps_demo_donate_gdp = pd.merge(gps_demo_donate, gdp, how='left', on="isocode")
gps_demo_donate_gdp.shape

(76, 73)

In [19]:
# gps_demo_donate_gdp[gps_demo_donate_gdp.filter(regex='.*_gdp', axis=1).isna().all(axis=1)]
gps_demo_donate_gdp.head()

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust,2019,2018,...,2010_gdp,2011_gdp,2012_gdp,2013_gdp,2014_gdp,2015_gdp,2016_gdp,2017_gdp,2018_gdp,2019_gdp
0,Afghanistan,AFG,-0.201360,0.120764,0.289641,0.254712,0.166455,0.315964,2.85,2.97,...,1.585657e+10,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.936264e+10,2.019176e+10,1.948438e+10,1.910135e+10
1,Algeria,DZA,0.059815,0.391530,-0.598255,0.254901,-0.009968,-0.177371,4.01,3.50,...,1.612078e+11,2.000195e+11,2.090603e+11,2.097548e+11,2.138088e+11,1.659794e+11,1.600338e+11,1.673894e+11,1.737580e+11,1.699882e+11
2,Argentina,ARG,-0.229308,0.041503,0.159679,-0.140457,0.027347,-0.206733,7.02,7.02,...,4.236274e+11,5.301633e+11,5.459824e+11,5.520251e+11,5.263197e+11,5.947493e+11,5.575314e+11,6.426959e+11,5.198715e+11,4.496634e+11
3,Australia,AUS,0.657004,0.137137,0.069660,0.022190,0.155331,0.298973,9.09,9.09,...,1.146138e+12,1.396650e+12,1.546152e+12,1.576184e+12,1.467484e+12,1.351694e+12,1.208847e+12,1.330136e+12,1.433904e+12,1.392681e+12
4,Austria,AUT,0.608285,-0.061829,0.161047,-0.055415,-0.035363,0.154628,8.29,8.29,...,3.918927e+11,4.311203e+11,4.094252e+11,4.300687e+11,4.419961e+11,3.818176e+11,3.952277e+11,4.183162e+11,4.555083e+11,4.463147e+11


### adding columns of funding / GDP * 100 by year

In [24]:
df_funding = convert_gdp_percent('funding', gps_demo_donate_gdp)
df_pledge = convert_gdp_percent('pledge', df_funding)

In [26]:
df_pledge.to_csv('data/gps_demo_donate_gdp.csv') 